In [14]:
%gui qt5
import numpy as np
import napari
import skimage
import imageio
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import skimage.feature as ft

import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px
import os
from os import path

import pandas as pd
import numpy as np
from ipywidgets import interactive, HBox, VBox

from skimage.feature import peak_local_max
from skimage import data, img_as_float
import tifffile
import glob
import cv2
import findmaxima.findmaxima as findmaxima

from ipywidgets import IntProgress
from IPython.display import display

## Load files and launch napari

In [70]:
files = glob.glob('./*/*projection.tif')

In [71]:
random_files = np.random.choice(files, size=len(files), replace=False)

In [75]:
viewer = napari.Viewer()

# Image analysis functions

### Utility functions

This will show an image in napari from a file name

In [72]:
def show_image(file_name, nap_viewer):
    data=tifffile.imread('./'+file_name)
    viewer.layers.clear()
    nap_viewer.add_image(data, channel_axis=0)
    nap_viewer.reset_view()
    for i in range(0,0):
        nap_viewer.layers[i].visible=False
    nap_viewer.layers[1].colormap='green'
    nap_viewer.layers[1].contrast_limits=[0,2000]
    return data
    

percentile_threshold takes an image, an SNR, and a percentile.  It calculates a stdev for all pixels below the percentile, and then returns a binary image with all pixels above the SNR*stdev set to 1.

In [73]:
def percentile_threshold(my_img, SNR, ptile):
    non_zeros=my_img>0
    mn=np.mean(my_img[non_zeros])
    ptile_value=np.percentile(my_img[non_zeros], ptile)
    stdev=np.std(my_img[non_zeros&(my_img<ptile_value)])
    mn=np.mean(my_img[non_zeros&(my_img<ptile_value)])
    return (my_img-mn)>(stdev*SNR)

peak_finder users Chris Wood's findmaxima implementation to find peaks in an image (it is a clone of ImageJ's find maxima).  It returns a binary image with all pixels that are local maxima set to 1.

In [74]:
def peak_finder(img, tol=300):
    x,y= findmaxima.findmaxima(img, tol)
    rtn = np.zeros(img.shape)
    rtn[x,y] = 255
    return rtn

DAPI takes the DAPI image, calls percentile_threshold, fills binary holes, finds the largest binary object, and filters out the edges up to border.  It returns a binary image with the largest DAPI object set to 1 shrunk by border.

In [76]:
def DAPI(DAPI_img, SNR=10, ptile=15, border=1, show=False):
    #smoothed=sp.ndimage.filters.gaussian_filter(DAPI_img, sigma=25)
    thresholded=sp.ndimage.binary_fill_holes(percentile_threshold(DAPI_img, SNR, ptile))

    labeled_image, labels=sp.ndimage.label(thresholded)
    indices, counts = np.unique(labeled_image[labeled_image>0], return_counts=True)
    if len(indices)<1: 
        DAPI_Mask = DAPI_img*0
    else:
        DAPI_Mask=(labeled_image==indices[np.argmax(counts)])

    edt = sp.ndimage.distance_transform_edt(DAPI_Mask)
    DAPI_Mask = edt>border
    
    if show:
        viewer.layers.clear()
        viewer.add_image(DAPI_img, colormap='magenta', name='DAPI_Img', blending='additive')
        viewer.add_image(thresholded*1000, colormap='magenta', name='Thresholded', blending='additive')
        viewer.add_image(DAPI_Mask*1000, colormap='green', name='DAPIMask', blending='additive')

    return DAPI_Mask

Filter the peak image to try and throw out tiny speckles and smooth, then apply Chris's fast findmax algorithm

In [168]:
def Peaks(peak_img, threshold, show=False):
    min_filtered=sp.ndimage.minimum_filter(peak_img/1.0, 5.0)
    min_gauss_filtered=sp.ndimage.gaussian_filter(min_filtered, sigma=0.8)

    peak_mask=peak_finder(min_gauss_filtered, threshold)
    
    if show:
        #clear_layers()
        viewer.add_image(peak_img, colormap='magenta', name='PeakImg', blending='additive')
        viewer.layers[-1].contrast_limits = [120,viewer.layers[-1].contrast_limits[1]/.04]
        viewer.add_image(min_filtered, colormap='magenta', name='MinFiltered', blending='additive')
        viewer.layers[-1].contrast_limits = [120,viewer.layers[-1].contrast_limits[1]/.04]
        viewer.add_image(min_gauss_filtered, colormap='green', name='MinGaussFiltered', blending='additive')
        viewer.layers[-1].contrast_limits = [120,viewer.layers[-1].contrast_limits[1]/.04]
        viewer.add_image(peak_mask*1000, colormap='green', name='Peaks', blending='additive')
    return peak_mask

Master processing function:  takes file name, finds the DAPI mask, finds the peaks in the peak_channels defined below with teh accompanying peak_thresholds.  If DAPI is letting in too much or too little:  adjust the DAPI call's SNR.

In [169]:
def process_file(fname, show=False):
    data=tifffile.imread(fname)
    
    images=[]
    peak_images=[]
    cur_data=[]
    DAPI_channel=3
    images.append(data[DAPI_channel,:,:])
    DAPI_Mask=DAPI(data[DAPI_channel,:,:], SNR=20, ptile=15, border=20, show=show)   
    images.append(DAPI_Mask>0)
    
    cur_data.append(fname)
    cur_data.append(np.sum(DAPI_Mask))

    for pc, pt in zip(peak_channels, peak_thresholds):
        peak_mask=Peaks(data[pc,:,:], pt, show)
        peak_mask=peak_mask*DAPI_Mask

        images.append(data[pc,:,:])
        images.append(peak_mask)
        cur_data.append(np.sum(peak_mask>0))

        if show:
            pts = np.array(np.where(viewer.layers[-1].data)).T
            viewer.add_points(pts, face_color='yellow')

    #images.append(data[0,:,:])
    #images.append(peak_images[0])
    #images.append(data[1,:,:])
    #images.append(peak_images[1])
    tifffile.imwrite(fname+'_processed.tif', np.asarray(images).astype('float32'), imagej=True)
    
    return cur_data

# Process images

### Configure peak finding

In [203]:
#For deep red:  0/600
#For red:  1/150
peak_channels=[0, 1]
peak_thresholds=[600, 300]

### Process a single file

In [204]:
#process_file(random_files[8], show=True)
process_file('./20211221_152540_452\Plate000_Well3_Object1.tif_projection.tif', show=True)

viewer.layers[1].visible=False
viewer.layers[2].visible=False
viewer.layers[0].contrast_limits=[90,500]

In [205]:
random_files[19]

'./20211221_152540_452\\Plate004_Well2_Object2.tif_projection.tif'

### Process a batch

In [206]:
all_data=[]

for i, f in enumerate(files):
    all_data.append(process_file(f))
    print([np.floor(i*100.0/len(files)), f])

[0.0, './20211221_152540_452\\Plate002_Well2_Object5.tif_projection.tif']
[0.0, './20211221_152540_452\\Plate003_Well2_Object2.tif_projection.tif']
[1.0, './20211221_152540_452\\Plate003_Well1_Object4.tif_projection.tif']
[2.0, './20211221_152540_452\\Plate003_Well2_Object6.tif_projection.tif']
[2.0, './20211221_152540_452\\Plate002_Well4_Object1.tif_projection.tif']
[3.0, './20211221_152540_452\\Plate003_Well4_Object4.tif_projection.tif']
[4.0, './20211221_152540_452\\Plate001_Well1_Object1.tif_projection.tif']
[5.0, './20211221_152540_452\\Plate004_Well3_Object2.tif_projection.tif']
[5.0, './20211221_152540_452\\Plate003_Well3_Object5.tif_projection.tif']
[6.0, './20211221_152540_452\\Plate001_Well3_Object7.tif_projection.tif']
[7.0, './20211221_152540_452\\Plate004_Well4_Object7.tif_projection.tif']
[8.0, './20211221_152540_452\\Plate004_Well4_Object2.tif_projection.tif']
[8.0, './20211221_152540_452\\Plate001_Well4_Object4.tif_projection.tif']
[9.0, './20211221_152540_452\\Plate003

[80.0, './20211221_152540_452\\Plate003_Well3_Object3.tif_projection.tif']
[81.0, './20211221_152540_452\\Plate000_Well3_Object8.tif_projection.tif']
[81.0, './20211221_152540_452\\Plate003_Well4_Object5.tif_projection.tif']
[82.0, './20211221_152540_452\\Plate002_Well1_Object5.tif_projection.tif']
[83.0, './20211221_152540_452\\Plate000_Well4_Object0.tif_projection.tif']
[83.0, './20211221_152540_452\\Plate000_Well3_Object10.tif_projection.tif']
[84.0, './20211221_152540_452\\Plate002_Well3_Object2.tif_projection.tif']
[85.0, './20211221_152540_452\\Plate000_Well2_Object2.tif_projection.tif']
[86.0, './20211221_152540_452\\Plate001_Well1_Object9.tif_projection.tif']
[86.0, './20211221_152540_452\\Plate001_Well1_Object10.tif_projection.tif']
[87.0, './20211221_152540_452\\Plate000_Well1_Object1.tif_projection.tif']
[88.0, './20211221_152540_452\\Plate001_Well2_Object0.tif_projection.tif']
[89.0, './20211221_152540_452\\Plate001_Well2_Object2.tif_projection.tif']
[89.0, './20211221_1525

Write out the results to csv

In [207]:
df=pd.DataFrame(all_data, columns=['File', 'Area', 'Density1', 'Density2'])
df.to_csv('Results.csv')

# Data Filtering

Load and map

In [208]:
df=pd.read_csv('Results.csv')
df['Area']=df['Area']*0.00078*0.00078

df['Density1']=df['Density1']/df['Area']
df['Density2']=df['Density2']/df['Area']
df['Batch']=df['File'].str.split('\\').str[0].str[2:]

In [209]:
df['Plate']=(df['File'].str.split('Plate').str[1].str[0:3])
df['Well']=df['File'].str.split('Well').str[1].str[0]
df['Object']=df['File'].str.split('Object').str[1].str.split('.tif').str[0]

df['PlateWell']=df['Plate']+'_'+df['Well']
df['Plate'] = df['Plate'].astype(int)
df['Well'] = df['Well'].astype(int)

In [210]:
mapper = pd.read_csv('Mappers.csv')
df = df.merge(mapper, on=['Batch', 'Plate', 'Well'])

### Explore and filter

In [214]:
def get_bad_list():
    if path.exists("Bad_List.csv"):
        bad_list = pd.read_csv('Bad_List.csv').drop(['Unnamed: 0'], axis=1)
    else:
        bad_list = pd.DataFrame(columns=['File'])
    return bad_list

In [215]:
bad_list = get_bad_list()

df = df[~(df['File'].isin(bad_list['File']))]

f=go.FigureWidget(
    px.box(df, x='Slide No.', y='Density2', hover_data=['File'], points='all', height=600)
    )
def selection_fn(trace,points,selector):
    
    if (len(points.point_inds)>0):
        
        global bad_list
        
        file = f.data[points.trace_index]['customdata'][points.point_inds[-1]][0]
        #bad_list = bad_list.append(pd.DataFrame([[file]], columns=bad_list.columns))
        bad_list = pd.concat([bad_list, pd.DataFrame([[file]], columns=bad_list.columns)])
        bad_list.to_csv('Bad_List.csv')
        print(file)
     
                
def click_fn(trace, points, state):
    
    if (len(points.point_inds)>0):
        print(f.data[points.trace_index]['customdata'][points.point_inds[-1]])
        file = f.data[points.trace_index]['customdata'][points.point_inds[-1]][0]
        show_image(file+'_processed.tif', viewer)
        
        for a in range(0,len(viewer.layers)):
            viewer.layers[a].visible = False
        #viewer.layers[-2].visible = True
        viewer.layers[-2].visible = True
        viewer.layers[-2].colormap = 'magenta'
        viewer.layers[-2].contrast_limits = [100,250]
        viewer.layers[1].contrast_limits = [0,1]
        
        pts = np.array(np.where(viewer.layers[-1].data)).T
        viewer.add_points(pts, face_color='yellow', name='Piwi')
        
        pts = np.array(np.where(viewer.layers[3].data)).T
        viewer.add_points(pts, face_color='yellow', name='H3p')
            
        #viewer.layers[6].colormap='magenta'
        #viewer.layers[5].colormap='yellow'
        #viewer.layers[6].contrast_limits = [90,500]

f.data[0].on_selection(selection_fn)

f.data[0].on_click(click_fn)

f

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'boxpoints': 'all',
              'custom…

# Plot  results

In [216]:
df = df.sort_values(by='Timepoint')

In [217]:
f=go.FigureWidget(
    px.box(df.reset_index(), 
           x='Secondary name', y='Density2', color='Primary name', points='all', height=800, hover_data=['index', 'File'])
    )

                
def click_fn(trace, points, state):
    
    if (len(points.point_inds)>0):
        idx = f.data[points.trace_index]['customdata'][points.point_inds[-1]][0]
        print(idx)
        file = df.loc[idx,'File']
        show_image(file+'_processed.tif', viewer)
        
        for a in range(0,len(viewer.layers)):
            viewer.layers[a].visible = False
            
        viewer.layers[2].colormap='magenta'
        viewer.layers[3].colormap='yellow'
        viewer.layers[4].colormap='magenta'
        viewer.layers[5].colormap='yellow'
        
        viewer.layers[4].visible = True
        print(file)
        viewer.layers[2].contrast_limits = [90,2000]
        viewer.layers[4].contrast_limits = [90,2000]
        viewer.layers[2].name = 'H3p'
        viewer.layers[4].name = 'Piwi'

        pts = np.array(np.where(viewer.layers[5].data)).T
        viewer.add_points(pts, face_color='yellow', name='Piwi Spots', visible=True)
        
        pts = np.array(np.where(viewer.layers[3].data)).T
        viewer.add_points(pts, face_color='yellow', name='H3p Spots', visible=False)

for a in f.data:
    a.on_click(click_fn)

f

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'boxpoints': 'all',
              'custom…

126
./20211221_152540_452\Plate000_Well4_Object0.tif_projection.tif
76
./20211221_152540_452\Plate001_Well4_Object4.tif_projection.tif
84
./20211221_152540_452\Plate001_Well2_Object5.tif_projection.tif
3
./20211221_152540_452\Plate002_Well2_Object3.tif_projection.tif
105
./20211221_152540_452\Plate004_Well2_Object1.tif_projection.tif


In [123]:
df.to_csv('Analysis.csv')

In [223]:
fig = px.box(df.reset_index(), x='Timepoint', y='Density2', color='RNAi', points='all', 
       height=800, hover_data=['index', 'File', 'Color'])
fig.write_image('Results.pdf')
fig